### Configure a Seamless outgoing port at localhost:8222

In [1]:
website="http://localhost:5813"
websocketserver="ws://localhost:5138"

import os
os.environ["SEAMLESS_COMMUNION_ID"] = "docking-clustering"
os.environ["SEAMLESS_COMMUNION_OUTGOING"] = "8222"
import json
import seamless
seamless.communionserver.configure_master(
    transformer_job=True,
    transformer_result=True,
    transformer_result_level2=True
)

### Use [Redis](https://redis.io/) as a cache, and as a store for new results

In [2]:
redis_sink = seamless.RedisSink()
redis_cache = seamless.RedisCache()

In [3]:
from seamless.highlevel import Context, Cell, Transformer
ctx = Context()

Set up a communion outgoing port 8222


### Load Seamless graph, previously auto-generated from the [Snakemake file](../../../edit/tests/docking/Snakefile)

In [4]:
graph = json.load(open("snakegraph.seamless"))
ctx = seamless.highlevel.load_graph(graph)

### Load docking input files and bind them to the graph's virtual file system

In [5]:
# HACK: Keep the large pairwise lrmsd file out of the virtual file system
ctx.pw_lrmsd = Cell()
file = "docking-result-pairwise-lrmsd.txt"
print(file)
data = open(file).read()
ctx.pw_lrmsd = data
ctx.jobs.cluster_struc.inputfile_pairwise_lrmsd = ctx.pw_lrmsd

inputs = (
    "receptor.pdb",
    "ligand.pdb",
    "receptor-bound.pdb",
    "ligand-bound.pdb",
    "docking-result.dat",
    #"docking-result-pairwise-lrmsd.txt"
)
for file in inputs:
    print(file)
    data = open(file).read()
    ctx.filesystem[file] = data


docking-result-pairwise-lrmsd.txt
receptor.pdb
ligand.pdb
receptor-bound.pdb
ligand-bound.pdb
docking-result.dat


### Set up initial values. This will start the computation

In [6]:
ctx.clustering_cutoff = 10
ctx.clustering_cutoff.celltype = "text"
ctx.selected_cluster = 6
ctx.selected_cluster.celltype = "text"
ctx.filesystem["params/cluster-cutoff"] = ctx.clustering_cutoff
ctx.filesystem["params/selected-cluster"] = ctx.selected_cluster

### Set up a read-out of the results

In [7]:
ctx.pdb_backbone = ctx.filesystem["outputs/selected-cluster-aligned-backbone.pdb"]
ctx.pdb_sidechain = ctx.filesystem["outputs/selected-cluster-aligned-sidechains.pdb"]
ctx.lrmsd = ctx.filesystem["outputs/selected-cluster.lrmsd"]
ctx.irmsd = ctx.filesystem["outputs/selected-cluster.irmsd"]
ctx.fnat = ctx.filesystem["outputs/selected-cluster.fnat"]
ctx.capri_stars = ctx.filesystem["outputs/selected-cluster.stars"]
ctx.docking_score = ctx.filesystem["outputs/selected-cluster.ene"]
ctx.rank = ctx.filesystem["outputs/selected-cluster.rank"]
ctx.cluster_size = ctx.filesystem["outputs/selected-cluster.size"]


### Wait until execution has finished

In [8]:
ctx.equilibrate()

set()

### Share the inputs and the results over the web

In [9]:
cells = (
 ctx.clustering_cutoff,
 ctx.selected_cluster,      
 ctx.pdb_backbone,
 ctx.pdb_sidechain,
 ctx.lrmsd,
 ctx.irmsd,
 ctx.fnat,
 ctx.capri_stars,
 ctx.docking_score,
 ctx.rank,
 ctx.cluster_size
)
for cell in cells:
    cell.celltype = "text"
    cell.share()
ctx.equilibrate()

Opened the seamless share update server at port 5138
Opened the seamless REST server at port 5813


set()

In [10]:
from IPython.display import Markdown
Markdown("""
### The inputs and results are now interactively shared over the web. For example: 
- [Clustering cutoff]({web}/ctx/clustering_cutoff)
- [backbone PDB]({web}/ctx/pdb_backbone)
- [full PDB]({web}/ctx/pdb_sidechain) (generated by Oscar-star)
- [Docking score]({web}/ctx/docking_score)
- [CAPRI stars]({web}/ctx/capri_stars)
""".format(web=website))


### The inputs and results are now interactively shared over the web. For example: 
- [Clustering cutoff](http://localhost:5813/ctx/clustering_cutoff)
- [backbone PDB](http://localhost:5813/ctx/pdb_backbone)
- [full PDB](http://localhost:5813/ctx/pdb_sidechain) (generated by Oscar-star)
- [Docking score](http://localhost:5813/ctx/docking_score)
- [CAPRI stars](http://localhost:5813/ctx/capri_stars)


### Set up Jupyter widgets to control the input and display the results

In [11]:
# Some helper functions
import ipywidgets as widgets
import traitlets
from collections import OrderedDict
from functools import partial


def connect(cell, widget):
    t = cell.traitlet()
    if isinstance(widget, widgets.Label):
        traitlets.dlink((t, "value"), (widget, "value"), lambda v: "" if v is None else str(v) )
    else:
        widget.value = t.value
        traitlets.link((t, "value"), (widget, "value"))


In [12]:


w = {
    "cutoff": widgets.BoundedFloatText(min = 0.5, max=50, step=0.5, description = "Clustering cutoff"),
    "sel": widgets.BoundedIntText(min = 1, max=100, step=0.5, description = "Selected cluster"),
    "irmsd": widgets.Label(description = "Interface RMSD"),
    "lrmsd": widgets.Label(description = "Ligand RMSD"),
    "fnat": widgets.Label(description = "Fraction of native contacts"),
    "stars": widgets.Label(description = "CAPRI stars"),
    "score": widgets.Label(description = "Docking score"),
    "rank": widgets.Label(description = "Docking rank"),
    "cluster_size": widgets.Label(description = "Cluster size"),
    
}

connect(ctx.selected_cluster, w["sel"])
connect(ctx.clustering_cutoff, w["cutoff"])
connect(ctx.irmsd, w["irmsd"])
connect(ctx.lrmsd, w["lrmsd"])
connect(ctx.fnat, w["fnat"])
connect(ctx.capri_stars, w["stars"])
connect(ctx.docking_score, w["score"])
connect(ctx.rank, w["rank"])
connect(ctx.cluster_size, w["cluster_size"])

a = OrderedDict()
a["Input"] =  w["sel"], w["cutoff"]
a["CAPRI evaluation"] = w["irmsd"], w["lrmsd"], w["fnat"], w["stars"]
a["Docking statistics"] = w["rank"], w["score"], w["cluster_size"]

tab = widgets.Tab()
for k,v in a.items():
    accordion = widgets.Accordion()
    for ww in v:
        accordion.children = accordion.children + (ww,)        
        accordion.set_title(len(accordion.children)-1, ww.description)
        ww.description = ""
    tab.children = tab.children + (accordion,)
    tab.set_title(len(tab.children)-1, k)

### Create viewer / representation / selection widgets for the PDBs

In [13]:
ctx.pdbs0 = {}
ctx.pdbs0.bound = open("receptor-bound.pdb").read() + open("ligand-bound.pdb").read()
ctx.pdbs0.pdb_backbone = ctx.pdb_backbone
ctx.pdbs0.pdb_sidechain = ctx.pdb_sidechain
ctx.pdbs = ctx.pdbs0
ctx.pdbs.celltype = "plain"
ctx.pdbs.share()

ctx.code = Context()
ctx.structurestate_class = Transformer() #Until Seamless supports modules at the high level
#ctx.structurestate_class.code.mount("../struclib/StructureState.py") # bug in Seamless
ctx.code.structurestate_class >> ctx.structurestate_class.code # KLUDGE
ctx.code.structurestate_class.mount("../struclib/StructureState.py", authority="file")
ctx.structurestate_schema = ctx.structurestate_class

ctx.load_pdbs = lambda structurestate_schema, pdbs: None
ctx.load_pdbs.structurestate_schema = ctx.structurestate_schema
ctx.load_pdbs.pdbs = ctx.pdbs
ctx.code.load_pdbs >> ctx.load_pdbs.code
ctx.code.load_pdbs.mount("../load_pdbs.py", authority="file")
ctx.struc_data = ctx.load_pdbs
ctx.equilibrate()

set()

In [14]:
ctx.visualization = ""
ctx.visualization.celltype = "text"
ctx.visualization.mount("visualization.txt", authority="file")
ctx.equilibrate()

set()

In [15]:
ctx.visualize = Transformer()
ctx.visualize.with_result = True
ctx.visualize.structurestate_schema = ctx.structurestate_schema
ctx.visualize.struc_data = ctx.struc_data
ctx.visualize.visualization = ctx.visualization
ctx.code.visualize >> ctx.visualize.code
ctx.code.visualize.mount("../visualize.py", authority="file")
ctx.visualize_result = ctx.visualize
ctx.equilibrate()

set()

In [16]:
ctx.ngl_representations = Cell()
ctx.ngl_representations = ctx.visualize_result.ngl_representations
ctx.ngl_representations.celltype = "plain"

ctx.selected = Cell()
ctx.selected = ctx.visualize_result.table
ctx.selected.celltype = "text"
ctx.selected.mimetype = "html"
ctx.equilibrate()

set()

In [17]:
ctx.visualization.share()
ctx.pdbs.share()
ctx.ngl_representations.share()
ctx.selected.share()
ctx.equilibrate()

set()

In [27]:
import nglview as nv
import functools
view = nv.NGLWidget()
view.components = {}

  
def ngl_set_representations(representations):
    if representations is None:
        return
    for code, representation in representations.items():
        if code not in view.components:
            continue
        comp = view.components[code]
        comp.set_representations(representation)

        
def ngl_load_pdbs(pdbs): 
    import seamless
    checksum = seamless.get_dict_hash(pdbs)
    if getattr(ngl_load_pdbs, "_checksum", None) == checksum:
        return
    ngl_load_pdbs._checksum = checksum
    for code, comp in view.components.items():
        try:
            view.remove_component(comp)
        except IndexError: #some bug in NGLWidget?
            pass
    view.components.clear()
    view.clear()
    if pdbs is None:
        return
    for code, pdb in pdbs.items():
        struc = nv.TextStructure(pdb,ext="pdb")
        view.components[code] = view.add_component(struc)
    ngl_set_representations(ctx.ngl_representations.value)

t=ctx.pdbs.traitlet()
observer1 = t.observe(lambda change: ngl_load_pdbs(change["new"]), "value")
t=ctx.ngl_representations.traitlet()
observer2 = t.observe(lambda change: ngl_set_representations(change["new"]), "value")

In [19]:
import ipywidgets, traitlets
selected = ipywidgets.HTML()
t = ctx.selected.traitlet()
_ = traitlets.directional_link((t, "value"), (selected, "value"), transform=lambda v: v if v is not None else "")

In [20]:
t = ctx.visualization.traitlet()
visualization = ipywidgets.Textarea()
_ = traitlets.directional_link(
    (t, "value"), (visualization, "value"), 
    transform=lambda v: v if v is not None else ""
)
visualization.layout.min_width = "800px"
visualization.rows = 10
vis_button = ipywidgets.Button(description="Update visualization")
def on_click(_):
    t.value = visualization.value
vis_button.on_click(on_click)

### Display the widgets

In [21]:
from IPython.display import display
display(tab)

Tab(children=(Accordion(children=(BoundedIntText(value=6, min=1, step=0), BoundedFloatText(value=10.0, max=50.0, min=0.5, step=0.5)), _titles={'0': 'Selected cluster', '1': 'Clustering cutoff'}), Accordion(children=(Label(value='0.994'), Label(value='1.8'), Label(value='0.72'), Label(value='3')), _titles={'0': 'Interface RMSD', '1': 'Ligand RMSD', '2': 'Fraction of native contacts', '3': 'CAPRI stars'}), Accordion(children=(Label(value=''), Label(value=''), Label(value='')), _titles={'0': 'Docking rank', '1': 'Docking score', '2': 'Cluster size'})), _titles={'0': 'Input', '1': 'CAPRI evaluation', '2': 'Docking statistics'})

3220 atoms selected
0 atoms selected
384 atoms selected
889 atoms selected
0 atoms selected
356 atoms selected
846 atoms selected
72 atoms selected
3221 atoms selected
0 atoms selected
384 atoms selected
889 atoms selected
0 atoms selected
356 atoms selected
847 atoms selected
72 atoms selected


In [28]:
view

NGLWidget()

In [26]:
display(visualization)
vis_button

Textarea(value='select all\nshow_as cartoon\nshow licorice\nselect obj=="bound" and chain=="A"\ncolor magenta\nselect obj=="pdb_backbone" and chain=="A"\ncolor yellow\nselect obj=="pdb_sidechain" and chain=="A"\ncolor yellow\nselect obj=="bound" and chain=="A"\ncolor blue\nselect obj=="pdb_backbone" and chain=="B"\ncolor green\nselect obj=="pdb_sidechain" and chain=="B"\ncolor green\nselect resid > 20 and resid < 21\nshow spacefill', layout=Layout(min_width='800px'), rows=10)

Button(description='Update visualization', style=ButtonStyle())

3221 atoms selected
0 atoms selected
384 atoms selected
889 atoms selected
0 atoms selected
356 atoms selected
847 atoms selected
72 atoms selected
3220 atoms selected
0 atoms selected
384 atoms selected
888 atoms selected
0 atoms selected
356 atoms selected
847 atoms selected
34 atoms selected


In [24]:
selected

HTML(value='')

In [25]:
Markdown("""
# Observable Notebook

### The PDB can also be visualized (and its visualization edited) using Observable Notebook
https://observablehq.com/@sjdv1982/struclib-viewer-gui?RESTSERVER=%{website}&WEBSOCKETSERVER={websocketserver}
""".format(website=website,websocketserver=websocketserver))


# Observable Notebook

### The PDB can also be visualized (and its visualization edited) using Observable Notebook
https://observablehq.com/@sjdv1982/struclib-viewer-gui?RESTSERVER=%http://localhost:5813&WEBSOCKETSERVER=ws://localhost:5138
